<a href="https://colab.research.google.com/github/wonjae124/Pytorch-tutorial/blob/main/torch_autograd_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import torch, torchvision

In [76]:
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1,1000)

In [77]:
prediction = model(data)

In [78]:
prediction.shape

torch.Size([1, 1000])

In [79]:
loss = (prediction-labels).sum()
loss.backward()

In [80]:
optim = torch.optim.SGD(model.parameters(), lr = 1e-2, momentum = 0.9)

In [81]:
optim.step()

# Autograd에서 미분

In [82]:
a = torch.tensor([2.,3.], requires_grad=True)
b = torch.tensor([6.,4.],requires_grad=True)

In [83]:
Q = 3*a**3 - b**2

In [84]:
Q

tensor([-12.,  65.], grad_fn=<SubBackward0>)

In [85]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad) 

In [86]:
Q

tensor([-12.,  65.], grad_fn=<SubBackward0>)

# directed acycli graph에서 제외하기

In [87]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad = True)

a = x+y
print(f"Does 'a' requires gradienets? :{a.requires_grad}")

b = x+z
print(f"Does 'b' require gradients? : {b.requires_grad}")

Does 'a' requires gradienets? :False
Does 'b' require gradients? : True


# 변화도를 계산하지 않는 매개변수를 고정시키기. Frozen parameter
# Fine Tuning

In [88]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
  param.requires_grad = False

In [89]:
model.fc = nn.Linear(512, 10)

optimizer = optim.SGD(model.parameters(), lr = 1e-2, momentum = 0.9) 

# Saved tensor

In [91]:
x = torch.randn(5, requires_grad = True)
y = x.pow(2)
print(x.equal(y.grad_fn._saved_self))
print(x is y.grad_fn._saved_self)

True
True


In [92]:
print(x)

tensor([ 1.0902, -0.5721,  0.1925,  0.3065, -0.9836], requires_grad=True)


In [93]:
print(y)

tensor([1.1886, 0.3272, 0.0371, 0.0939, 0.9675], grad_fn=<PowBackward0>)


In [94]:
y.grad_fn._saved_self

tensor([ 1.0902, -0.5721,  0.1925,  0.3065, -0.9836], requires_grad=True)

In [95]:
x = torch.rand(5, requires_grad = True)
y = x.exp()
print(y.equal(y.grad_fn._saved_result)) # 같은 차원과 element를 가지고 있을 때 torch.equal()사용
print(y is y.grad_fn._saved_result) #같은 객체인지를 볼 때 is 사용

True
False


In [96]:
print(y == y.grad_fn._saved_result)

tensor([True, True, True, True, True])


In [97]:
print(y)

tensor([2.0957, 2.3128, 1.1693, 1.4164, 1.8381], grad_fn=<ExpBackward0>)


In [98]:
print(y.grad_fn._saved_result)

tensor([2.0957, 2.3128, 1.1693, 1.4164, 1.8381], grad_fn=<ExpBackward0>)


# Multithreaded Autograd

In [99]:
def train_fn():
  x = torch.ones(5, 5, requires_grad = True)
  y = (x+3)*(x+4)*0.5

  y.sum().backward()

threads = []

for _ in range(10):
  p = threading.Thread(target = train_fn, args = ())
  p.start()
  threads.append(p)

for p in threads:
  p.join()

NameError: ignored

# Autograd & Variable
# https://do-hansung.tistory.com/16


In [100]:
import torch
from torch.autograd import Variable
a = torch.ones(2,2)
print(a)

tensor([[1., 1.],
        [1., 1.]])


In [101]:
a = Variable(a, requires_grad=True)
print(a)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [102]:
print(a.data)

tensor([[1., 1.],
        [1., 1.]])


In [103]:
print(a.grad)

None


In [104]:
print(a.grad_fn)

None


In [105]:
b = a + 2
print(b)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [106]:
c = b**2
print(c)

tensor([[9., 9.],
        [9., 9.]], grad_fn=<PowBackward0>)


In [107]:
out = c.sum()
print(out)

tensor(36., grad_fn=<SumBackward0>)


In [108]:
out.backward() #backward 함수가 grad를 채워준다.
print(a.grad)

tensor([[6., 6.],
        [6., 6.]])


In [110]:
print(a.grad_fn) #a가 직접 수행한 연산이 없기 때문에 None

None


In [111]:
b.data

tensor([[3., 3.],
        [3., 3.]])

In [113]:
print(b.grad)

None


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


In [116]:
print(b.grad_fn) #b는 이전에 b = a+2로 기울기가 있는 연산(AddBackward)를 한 상태임

In [118]:
print(c.data)

tensor([[9., 9.],
        [9., 9.]])


In [119]:
print(c.grad)

None


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


In [120]:
print(c.grad_fn)

In [121]:
print(out.data)

tensor(36.)


In [122]:
print(out.grad)

None


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


In [123]:
print(out.grad_fn)

In [124]:
x = torch.ones(3)
x = Variable(x, requires_grad = True)
y = x**2
z = y*3
print(z)

tensor([3., 3., 3.], grad_fn=<MulBackward0>)


In [125]:
grad = torch.Tensor([0.1, 1, 10])
z.backward(grad) #z=3x^2을 x에 대해 편미분 하면 6x가 된다. 이는 결국 x를 업데이트 하기 위함을 뜻 한다.

In [126]:
print(x.data)

tensor([1., 1., 1.])


In [127]:
print(x.grad) # 이전의 gradient 값에 차례로 6을 곱한다.

tensor([ 0.6000,  6.0000, 60.0000])


In [128]:
print(x.grad_fn)

None


In [129]:
print

<function print>